In [ ]:
import sctoolbox
from sctoolbox.utilities import bgcolor

# Batch effect correction and comparisons
<hr style="border:2px solid black"> </hr>

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
%bgcolor PowderBlue

# Choose normalization method
norm_methods = ['tfidf', 'total']  # can be 'tfidf' and/or 'total'

# Set number of neighbors
n_neighbors = 15  #Default=15

# UMAP related settings 
metacol = 'sample'

# batch correction: If True, several batch correction methods will be performed,
# you can choose the best one after
batch_column = "sample"
perform_batch_correction = True
batch_methods = ["bbknn", "harmony"] # "mnn", "scanorama"
threads = 8

<hr style="border:2px solid black"> </hr>

## Set up

In [ ]:
import sctoolbox.tools as tools
import sctoolbox.plotting as pl
import sctoolbox.utils as utils
import scanpy as sc
import episcanpy as epi
import numpy as np
import matplotlib.pyplot as plt
utils.settings_from_config("config.yaml", key="03")

sc.set_figure_params(vector_friendly=True, dpi_save=600, scanpy=False)

----------------

## Load anndata 

In [ ]:
# probably updated in future
adata = utils.load_h5ad("anndata_2.h5ad")
display(adata)

----------

## Normalization

In [ ]:
normalizations = tools.normalize_adata(adata, norm_methods)

--------

## Show PC embedding per method

In [ ]:
%bgcolor PowderBlue

# number of PCs shown within the heatmap
n_pcs_heatmap = 15

In [ ]:
_ = pl.compare_embeddings(list(normalizations.values()), adata_names=list(normalizations.keys()),
                          var_list=[batch_column], embedding="pca")

In [ ]:
# PCA correlations with obs and var variables
for method, adata_norm in normalizations.items():
    
    _ = pl.plot_pca_correlation(adata_norm, 
                                n_components=n_pcs_heatmap,
                                which="obs", 
                                title=f"Normalization method = {method}\nCorrelation of .obs columns with PCA loadings",
                                save=f"PCA_{method}_correlation_obs.pdf")
    
    _ = pl.plot_pca_correlation(adata_norm, 
                                n_components=n_pcs_heatmap,
                                which="var", 
                                title=f"Normalization method = {method}\nCorrelation of .var columns with PCA loadings",
                                save=f"PCA_{method}_correlation_var.pdf")

---------------------

## Subset PCs

In [ ]:
%bgcolor PowderBlue

# Whether PCs should be filtered
subset_pcs = True

corr_thresh = 0.5  # PCs with an absolut correlation above this will be filtered
perc_thresh = 50  # Top percentile of PCs that should be kept
filter_methods = ['variance', 'correlation']  # propose PCs based on the provided methods
basis = 'obs'  # base correlation on obs or var

In [ ]:
if subset_pcs:
    for key, adata_normed in normalizations.items():
        
        selected_pcs = tools.propose_pcs(anndata=adata_normed,
                                 how=filter_methods,
                                 corr_thresh=corr_thresh,
                                 perc_thresh=perc_thresh,
                                 corr_kwargs={'method': 'spearmanr', 'which': basis})
        
        # Plot and select number of PCs
        _, ax = plt.subplots()
        ax.set_title(key)
        _ = pl.plot_pca_variance(adata_normed, 
                                 save=key + "_PCA_variance_selected.pdf",selected=selected_pcs,
                                 n_pcs=50,
                                 n_thresh=max(selected_pcs),
                                 corr_plot='spearmanr',
                                 corr_thresh=corr_thresh,
                                 corr_on=basis, 
                                 ax=ax)
        tools.subset_PCA(adata_normed, select=selected_pcs)
        
        normalizations[key] = adata_normed

--------

## Calc standard neighbors and umap for each adata

In [ ]:
for adata in normalizations.values():
    sc.pp.neighbors(adata, n_neighbors=n_neighbors, method='umap', metric='euclidean')

In [ ]:
tools.wrap_umap(normalizations.values())

------------------------

## Compare between normalizations

In [ ]:
#Plot the overview of batch correction methods
_ = pl.anndata_overview(normalizations, 
                        plots=["PCA", "PCA-var", "UMAP"],
                        color_by=["n_features"], 
                        output=None)

In [ ]:
%bgcolor PowderBlue

# Choose the final normalization method
norm_method = "tfidf"

adata = normalizations[norm_method]

-----------

## Plot final PCA

In [ ]:
sc.pl.pca_overview(adata, color=['n_features'], show=False)

_ = pl.plot_pca_correlation(adata, which="var", 
                            title=f"Normalization method = {method}\nCorrelation of .var columns with PCA loadings",
                            save=f"PCA_{method}_correlation_var.pdf")

-----------

## Batch correction

In [ ]:
if perform_batch_correction:
    batch_corrections = tools.wrap_corrections(adata, 
                                                  batch_key=batch_column,
                                                  methods=batch_methods)
else:
    batch_corrections = {"uncorrected": adata}

In [ ]:
#Run standard umap for all adatas
tools.wrap_umap(batch_corrections.values(), threads=threads)

In [ ]:
# Should preliminary clustering be performed?
do_clustering = True #True or False

In [ ]:
#Perform additional clustering if it was chosen
color_by = []
if do_clustering:
    for adata in batch_corrections.values():
        sc.tl.leiden(adata, 0.1)
    color_by.append("leiden")

In [ ]:
# Calculate LISI scores for batch
tools.wrap_batch_evaluation(batch_corrections, batch_key=batch_column, threads=threads, inplace=True)

In [ ]:
#Plot the overview of batch correction methods
adata.obs[batch_column] = adata.obs[batch_column].astype("category") #ensure that batch column is a category

_ = pl.anndata_overview(batch_corrections, color_by=color_by + [batch_column], 
                        output=None)

In [ ]:
%bgcolor PowderBlue

#Choose an anndata object to proceed
batch_name = "bbknn"

adata_corrected = batch_corrections[batch_name]

---------

## Save anndata

In [ ]:
#Saving the data
adata_output = "anndata_3.h5ad"
utils.save_h5ad(adata, adata_output)